%load_ext autoreload             # loads the autoreload package into ipython kernel
%autoreload 2                    # sets autoreload mode to automatically reload modules when they change
%config IPCompleter.greedy=True  # enables tab completion# Linear Solver for Battery Control in PV installation

This notebook attempts to use the CVXPY library to solve the 
optimisation problem posed in [1]. It follows the notation
of the paper.

[1] https://ieeexplore.ieee.org/iel7/5165391/5433168/07470517.pdf

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
# Importing relevant libraries
import cvxpy as cp
import numpy as np

In [ ]:
# Loading data
from solara.constants import PROJECT_PATH
load_data = np.loadtxt(PROJECT_PATH + "/data/solar_trace_data/load_mid1.txt", delimiter=",")
pv_data = np.loadtxt(PROJECT_PATH + "/data/solar_trace_data/PV_5796.txt", delimiter=",")

## 1. Setting up the problem

In [ ]:
# Setting all the variables

# Given variables
B = 10 # Capacity of ESD (kWh)
MD = 0.1 # ESD maximum discharge and charge fractions
MC = 0.9 # See above
alpha_c = 0.33 # Charge (discharge) rate limits per unit of storage (kW/kWh)
alpha_d = 1.67 # See above
eta_c = 0.85 # Charge (discharge) efficiency. Both are≤1
eta_d = 1 # # See above
U = B*MD
pi_b = 0.14 #0.14 # Base price per unit of energy purchased ($/kWh)
pi_d = 1.00 # Demand price penalty per unit of energy purchased with power demand exceeding Γ($/kWh)
Gamma = np.percentile(load_data, 80) # Threshold above which the demand price is paid (kW)
T_u = 1 # Time slot duration
T_h = 24 # Time horizon (hours)
p_bar = 0.12 # Price per unit of energy sold at time t ($/kWh)

# Given variables from data set
num_timesteps = T_h
start = 24*99
P_L = load_data[start:start+num_timesteps] #np.random.randn(num_timesteps) # Load at time t (kW)
P_S = pv_data[start:start+num_timesteps] #np.random.randn(num_timesteps) # Power generated by solar panels at timet(kW)


# Variables that being optimised over
P_dir = cp.Variable(num_timesteps) # Power flowing directly from PV and grid to meet the load or be sold at time t (kW)
P_c = cp.Variable(num_timesteps) # Power used to charge the ESD at time t (kW)
P_d = cp.Variable(num_timesteps) # Power from the ESD at time t (kW)
P_g = cp.Variable(num_timesteps) # Power drawn from the grid at time t (kW)
P_sell = cp.Variable(num_timesteps) # Power sold to the grid at timet(kW)
P_over = cp.Variable(num_timesteps) #  Purchased power that exceeds Γ at time t (not in notation table)
I = cp.Variable(num_timesteps, boolean=True) # Boolean complies with contraint Eq (20)

# Implicitly defined variable (not in paper in "given" or "optimized over" set of variables)
E_ESD = cp.Variable(num_timesteps+1) # the  energy  content  of  the  ESD  at  the  beginning  of  interval t

In [ ]:
constraints = [0 <= P_g, # from Equation (13)
               0 <= P_dir,
               0 <= P_sell,
               P_dir + P_d == P_L + P_sell, # from Equation (14)
               E_ESD[0] == B*MD, # Eq (15)
               E_ESD[1:] == E_ESD[:-1] + eta_c*P_c*T_u - (1/eta_d) * P_d * T_u, # Eq (16)
               0 <= P_c + P_dir, # Eq (17)
               P_c + P_dir <= P_S + P_g, # Eq (17)
               0 <= P_c, # Eq (18)
               P_c <= I * B * alpha_c, # Eq (18)
               0 <= P_d, # Eq  (19)
               P_d <= (1 - I) * B * alpha_d, # Eq  (19)
               B*MD <= E_ESD, # Eq (21)
               E_ESD <= B*MC, # Eq (21)
               0 <= P_over, # Eq (23)
               P_g - Gamma <= P_over, # Eq (24)
               ]

In [ ]:
objective = cp.Minimize(cp.sum(pi_b*P_g + pi_d*P_over - cp.multiply(p_bar,P_sell)))

In [ ]:
prob = cp.Problem(objective, constraints)

## 2. Solving the problem

In [ ]:
result = prob.solve() #verbose=True)

## 3. Showing the resulting control

In [ ]:
P_sell.value

In [ ]:
P_c.value

In [ ]:
P_d.value

In [ ]:
E_ESD.value

In [ ]:
from solara import visualisation

visualisation.plot_battery_control(E_ESD.value[:24], P_S, P_L, save_path="control_plot_1.png")

## Reference code

In [ ]:
# Battery variables as per newer battery model

size = 1
kWh_per_cell = 0.011284
num_cells = size / kWh_per_cell

# parameters specified for an LNMC cell with operating range of 1 C
# charging and discharging
nominal_voltage_c = 3.8793
nominal_voltage_d = 3.5967
a1_slope = 0.1920
a1_intercept = 0.0
a2_slope = -0.4865
a2_intercept = kWh_per_cell * num_cells
eta_d = 1 / 0.9  # taking reciprocal so that we don't divide by eta_d
eta_c = 0.9942
alpha_d = (
    a2_intercept * 1
)  # the 1 indicates the maximum discharging C-rate
alpha_c = (
    a2_intercept * 1
)  # the 1 indicates the maximum charging C-rate

In [ ]:
import cvxpy as cp
import numpy as np

# Below is the standard tutorial from https://www.cvxpy.org/

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A@x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
result